In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import math
import pickle 
path = Path('./')
train_path = path.joinpath('train.csv')
test_path = path.joinpath('test.csv')
submission_path = path.joinpath('submission_popular.csv')
metadata_path = path.joinpath('item_metadata.csv')

In [2]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_meta = pd.read_csv(metadata_path)
df_submission = pd.read_csv(submission_path)

In [3]:
action_list = set(df_train['action_type'].values)
reference_list = set(df_train['reference'].values)
platform_list = set(df_train['platform'].values)
city_list = set(df_train['city'].values)
device_list = set(df_train['device'].values)
filter_list = set(df_train['current_filters'].values)
filter_list =  [x for x in filter_list if str(x) != 'nan']

action_list = list(action_list)
action_list.sort()

reference_list = list(reference_list)
reference_list.sort()

platform_list = list(platform_list)
platform_list.sort()

city_list = list(city_list)
city_list.sort()

device_list = list(device_list)
device_list.sort()

filter_list = list(filter_list)
filter_list.sort()

In [4]:
#reference embedding 보조 함수
items = df_meta['item_id'].values
properties = df_meta['properties'].values
all_items_test = []
for i in range(len(properties)):
    p_data = properties[i].split('|')
    all_items_test += p_data
set_items = set(all_items_test)

list_items = list(set_items)
list_items.sort()
def embedding_item_diction():
    all_items = {}
    for i in range(len(properties)):
        all_items[items[i]] = properties[i].split('|')
    embedding_items = {}
    #print(len(list_items))
    for keys in all_items.keys():
        item_embeddings = [0]*len(list_items)
        for i in range(len(list_items)):
            if list_items[i] in all_items[keys]:
                item_embeddings[i] = 1
        embedding_items[keys] = item_embeddings
    return embedding_items
reference_diction = embedding_item_diction()

In [7]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [241]:
#filter embedding 보조함수
all_filters = []
for i in range(len(filter_list)):
    if i>0:
        all_filters += filter_list[i].split('|')
all_filter_set = set(all_filters)
all_filter_list = list(all_filter_set)
all_filter_list.sort()
def filter_embedding(data, version):
    embedding_data = [0]*202
    if version == 1:
        datalist = data.split('|')
        for filters in datalist:
            if filters in all_filter_list:
                embedding_data[all_filter_list.index(filters)] = 1
        return embedding_data

In [237]:
all_filter_list.index('WiFi (Public Areas)')

ValueError: 'WiFi (Public Areas)' is not in list

In [239]:
len(all_filter_list)

202

In [9]:
def one_hot_encoding(number, length):
    return [int(i==number) for i in range(length)]

In [10]:
# 전체 action embedding
def embedding(data, action_index):
    if action_index == 0: #user_id dim 0
        return []
    
    if action_index == 1: # session_id dim 0
        return []
    
    if action_index == 2: # timestep dim 0
        return []
    
    if action_index == 3: #step dim 1
        return []
    
    if action_index == 4: #action_type dim 10
        #print(one_hot_encoding(action_list.index(data), 10))
        return one_hot_encoding(action_list.index(data), 10)
    
    if action_index == 5: #reference dim 157
        if RepresentsInt(data):
            return reference_diction[int(data)]
        else:
            return [0]*157
        
    if action_index == 6: #platform dim 55
        return one_hot_encoding(platform_list.index(data), 55)
    
    if action_index == 7: #city dim 0
        #return city_list.index(data)
        return []
    
    if action_index == 8: #device dim 3
        return one_hot_encoding(device_list.index(data), 3)
    
    if action_index == 9: #current_filters dim 202
        if(isinstance(data, float) and math.isnan(data) == True):
            return [0]*202
        else:
            return filter_embedding(data, 1)
        
    if action_index == 10: #impressions dim 157
        if(isinstance(data, float) and math.isnan(data) == True):
            return [0]*157
        else:
            impression_embedding = [0]*157
            impression_list = data.split('|')
            for impression in impression_list:
                try:
                    impression_embedding =sum_list(impression_embedding, reference_diction[int(impression)])
                    
                except:
                    pass
                    #print(int(impression))
            '''
            if(len(impression_embedding) < 3925):
                impression_embedding += [0]*(3925-len(impression_embedding))   
            '''
            return impression_embedding
            
        
    if action_index == 11: #prices dim 25
        if(isinstance(data, float) and math.isnan(data) == True):
            return [0]*25
        else:
            price_embedding = []
            for prices in data.split('|'):
                price_embedding.append(int(prices))
            if(len(price_embedding) < 25):
                price_embedding += [0]*(25-len(price_embedding))   
            return price_embedding

In [91]:
def MakeEmbeddingVector(data):

    item_vector = [] # item data embedding
    num_action = len(data)
    #time_delay_coef = 0.1
    reference_vector = []
    filter_vector = [0]*202
    current_reference  = ''
    reference_size = 0
    for i in range(num_action-1, -1, -1): #거꾸로
        timestep_vector = []
        if i != num_action-1 and RepresentsInt(data[i, 5]) and current_reference != data[i,5] and reference_size < 2:
            current_reference = data[1,5]
            if int(data[i, 5]) in reference_diction:
                reference_size += 1
                reference_vector += reference_diction[int(data[i, 5])]
        if(filter_vector == [0]*202):
            filter_vector = embedding(data[i,9], 9)
    reference_vector += [0]*(314-len(reference_vector))
    session_vector = embedding(data[-1, 4], 4) + reference_vector + embedding(data[-1, 6], 6) + embedding(data[-1, 8], 8) + filter_vector
    # 10+314+202+55+3 = 584

    #item_vector.append(reference_diction[int(data[-1,5])])
    temp_impressions = data[-1, 10].split('|')
    for i in range(len(temp_impressions)):
        if int(temp_impressions[i]) in reference_diction:
            item_vector.append(reference_diction[int(temp_impressions[i])])
        else:
            item_vector.append([0]*157)
    return session_vector, item_vector

In [68]:
# 데이터를 세션 단위로 나눔
def make_data(data): 
    data_split = []
    temp_data = []
    data_size = len(data)
    action_index = 1
    for i in range(data_size):
        if(data[i,action_index] == data[max(i-1,0),action_index]):
            temp_data.append(data[i])
        else:
            data_split.append(np.array(temp_data))
            temp_data = []
            temp_data.append(data[i])
        if(i == data_size-1):
            data_split.append(np.array(temp_data))
    return data_split

In [69]:
test_data = df_test.values

In [22]:
train_data = df_train.values

In [70]:
test_data_split = make_data(test_data)

In [71]:
def check_nan(data):
    datalength = len(data)
    output_data = []
    max_error = 0
    #nan_list = []
    for i in range(datalength):
        step_length = len(data[i])
        for j in range(step_length):
            if(data[i][j,4] == 'clickout item'):
                if(isinstance(data[i][j,5], float) and math.isnan(data[i][j,5]) == True):
                    output_data.append(data[i][:j+1,:])
                    #nan_list.append
                    #break;
    return output_data

In [72]:
test_data_preprocess = check_nan(test_data_split)

In [81]:
import numpy as np
import tensorflow as tf
data_num = len(test_data_preprocess)
train_embedding_dim= 584
item_embedding_dim = 157
latent_dim = 200
learning_rate = 1e-3


In [83]:
tf.reset_default_graph()
session_feature = tf.get_variable("session_feature", [train_embedding_dim, latent_dim], initializer=tf.random_normal_initializer())
item_feature = tf.get_variable("item_feature", [item_embedding_dim, latent_dim], initializer=tf.random_normal_initializer())
session_bias = tf.get_variable("session_bias", [train_embedding_dim], initializer=tf.random_normal_initializer())
item_bias = tf.get_variable("item_bias", [item_embedding_dim], initializer=tf.random_normal_initializer())

In [84]:
train_data = tf.placeholder(tf.float32, shape=(train_embedding_dim))
train_items  = tf.placeholder(tf.float32, shape=(None, item_embedding_dim))
train_index = tf.placeholder(tf.int32, [1])

In [85]:
scores = tf.matmul(tf.matmul(train_items, item_feature), tf.transpose(tf.matmul(tf.expand_dims(train_data,0), session_feature))) +  \
tf.matmul(tf.expand_dims(train_data,0), tf.expand_dims(session_bias,1)) +\
tf.matmul(train_items, tf.expand_dims(item_bias,1))

In [100]:
scores 

<tf.Tensor 'add_1:0' shape=(?, 1) dtype=float32>

In [86]:
loss = -tf.reduce_mean(tf.math.log(tf.sigmoid(scores[train_index[0]] - scores)))
loss2 = tf.reduce_mean(tf.sigmoid(scores- scores[train_index[0]]) + tf.sigmoid(tf.math.pow(scores,2)))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss2)

In [87]:
def get_MRS(score, idx):
    return (list(np.squeeze(np.argsort(score, axis = 0))).index(idx)+1)/np.shape(score)[0]

In [242]:

final_test_data = test_data_preprocess
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
max_mrs = 0
num_test = len(final_test_data)
saver = tf.train.Saver()
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, './saved_model/task1_ver2')
    step = 0
    mean_loss = 0
    mean_rs = 0
    data_number = 0
    submission_list = []
    submission_list.append(['user_id', 'session_id', 'timestamp', 'step', 'item_recommendations'])
    for i in range(num_test):

        batch_data, batch_items = MakeEmbeddingVector(final_test_data[i])
        batch_data  = np.asarray(batch_data)
        batch_items = np.asarray(batch_items)
        
        _score = sess.run(scores, feed_dict = {train_data : batch_data, train_items : batch_items})
        max_list = list(np.argsort(np.squeeze(_score)))
        max_list.reverse()
        #try:
        impression_list = final_test_data[i][-1,10].split('|')
        recommendations = [impression_list[max_list[i]] for i in range(len(max_list))]

        recommendations = ' '.join(recommendations)
        submission_list.append([final_test_data[i][-1,0], final_test_data[i][-1,1], final_test_data[i][-1,2], final_test_data[i][-1,3], recommendations])        
        if(i % 1000 ==0):
            print(submission_list[i+1])
        

INFO:tensorflow:Restoring parameters from ./saved_model/task1_ver2
['004A07DM0IDW', '1d688ec168932', 1541555799, 7, '3241426 399441 2249874 1050068 103357 1472107 3881554 127132 2033381 1724779 1167004 127128 2059240 127131 4491766 4359818 3135079 7997464 5162722 9030086 3442272 1963235 5838202 5834720 6827260']
['SCVAEQ7UCISG', 'b4fe7e9a56f66', 1541642397, 53, '1535525 127745 127713 2839044 489096 349036 4510026 4773566 3535138 2348444 1249719 9496738 5423974 1009031 2814611 8306368 3978000 9813914 11266490 11078934 8287404']
['LP1NXYZZ8D0D', 'e1e3bd1920c56', 1541623943, 4, '95565 94197 6943 17803 6951 6939 37445 37436 37434 37381 37446 37385 37358 1132391 6877 37448 6415842 40613 1690659 3184413 37392 1503731 132194 905071 16808']
['EH662X1GCU4P', '02b03083d9434', 1541557204, 63, '2319566 2026817 2552898 3084238 2570000 2318438 4109932 2729506 1656947 1998479 1859237 4089396 1045306 3865946 6570200 4051328 6542484 4010588 1056384 2363543 4004224 1457325 1656477 2368268 2853300']
['6J

['PJMZLWZUW79C', 'ebba3284e3086', 1541583738, 4, '321766 1021343 1021335 2023089 2100060 1021383 1021333 1021317 7097350 1021325 1520931 2270878 1021339 1021321 1198128 3548432 3548428 1198130 5923494 4531842 2338588 6918936 3546824 3502918 103547']
['H171NM8DB9P2', '0d8b338d5d94e', 1541690205, 3, '1049662 1588209 693426 2153528 1063748 693431 1342447 103540 106190 2431278 1552637 1955261 3546388 1121135 5498288 3545324 3545392 1162450 6306296 10494294 2105500 1197128']
['96KZJCBF1AWF', '6e7684b515691', 1541681556, 22, '2727852 6806560 921237 42065 2423428 4173522 5214176 5121038 9051714 42262 7384214 921315 42125 42097 9773310 2870052 2174748 106586 42105 42149 5773634 42207 4352682 42285 39740']
['1Q5ZQJ183NTX', '0fd3b2ed1e564', 1541666995, 3, '6321878 5817210 2141006 3754234 2173618 2281118 4534262 3184588 4844848 7775148 6623266 5862632 1541249 6066134 3150207 8970914 5710478 592056 1451293 3150326 1710923 66223 1827431 3491468 921867']
['SVXQBZM6CVX3', 'c85a83e043020', 1541601550,

['XRY21XIOE52I', 'dacbd99170709', 1541583137, 1, '2851320 3228242 3200300 3932110 3368464 2892823 2562552 4058896 3201612 3216814 3190466 3956876 4982014 4111876 8338534 4110224 3200606 5667170 3799262 4098624 3199559 10106434 7309986 8363698 7939930']
['R2S5ZSY29FAY', '3ee161c64ead0', 1541627089, 1, '1929355 16944 16837 16885 153597 1689815 2838056 150160 1504057 16886 153567 45832 2567806 3131976 16844 17530 1585753 921555 3872592 3883882 2941930 2632980 857986 2837104 8231876']
['IO0G7UQLR35I', 'ac61331ab5ae6', 1541605089, 9, '17741 17760 17738 17783 2927966 17630 17784 728921 2583176 5163040 894329 6038484 5426468 10452732 10435028']
['AUOO8WO1SJ6F', 'ed8648397788a', 1541632221, 13, '954607 2454120 2306908 2037563 1235860 2029563 1668647 1001561 1345387 2828701 1669635 3125416 1668645 2773238 5049246 4871050 3248926 3248796 2027905 5635240 6298944 9105866 8967078 9682788 9232996']
['4MF94VMB8DR8', '8e99b9040cda8', 1541577630, 8, '49846 11952 11933 99872 2179674 81853 50141 82322 12

['6IX162K40DEM', '4cfe02eaa6dad', 1541649360, 1, '44028 43520 44776 44625 101112 918365 1147637 2583790 44713 2013061 4879096 2125138 3367251 3868196 843436 5819280 3106902 3755020 4194916 6924230 3164815 3211754 7971300 9572864']
['ZY8YX9WL2WU2', '314305428d9f3', 1541631202, 26, '104677 4415314 150138 111534 4342016 623276 109351 4155306 104565 109353 4345728 104838 5066908 5435302 3497814 4304686 104809 3148916 104866 104917 5659850 1319947 104880 5660226 5493194']
['QKAIZGX2AAFB', '58356b878e5fc', 1541564627, 3, '97298 37733 13088 80263 1319616 96242 1240081 117816 3181779 5008542 3590640 4538834 2022759 486741 4088834 4029784 3998358 5816622 3845896 7118816 6368678 1055748 6367820 7180592 2654682']
['KM7K28SYQT8E', '0e19d90ac3480', 1541691173, 23, '99630 321911 45763 853981 110132 45434 130903 1909039 437296 484896 150043 99627 2648696 388681 45659 4070480 4979922 1148999 210481 1277337 1942279 1359740 1264968 6864410 2686309']
['A1TEM9UVB3SL', 'f65e743058cf9', 1541605551, 10, '674

['U4M8N6H4N7ZZ', '2c9d3d1ea32b7', 1541551699, 1, '2777588 2772624 2789694 5197254 2734815 3203744 2776093 2272740 6783114']
['MBZ32S9F5OXX', '7c58217df5a41', 1541641683, 9, '10373502']
['E196W1E1ST58', '3c752a91cd067', 1541612737, 1, '42615 42494 2734597 1633547 116194 5965860 6832348 3800950 1844305 1744505 2510728 3768628 42859 504971 42710 42571 684811 3043940 2030107 2351476 2012077 4879672 4690612 8440364 4658388']
['7LQ49ZWNVSE8', 'dca8fba60ead7', 1541579016, 3, '8621 8586 8521 8599 8568 488501 8620 1167514 8589 1283361 32727 8595 2793224 1258697 6723510 102087 8565 2386475 76759 2628508 77124 5181618 77487 8509414 8437378']
['0ENW2VIQT64W', '58e1d559bc44c', 1541701601, 3, '43835 44360 43552 44644 43606 843481 941294 43861 43451 347366 43373 43875 43580 44609 43924 43725 44603 1181148 43610 44078 44389 347361 1455805 1949661 6703868']
['TV5YNCN6931F', '7f30c958ae816', 1541619562, 15, '6275774 1051228 104080 7094758 7363952 2123858 3586012 567776 3395276 1561891 3822156 2018183 10

['U6PJWHEL5TLU', '9c4931b2901bd', 1541599621, 3, '1321090 880913 47191 47101 3218556 47158 18427 12712 3755574 12695 5722432 365001 46272 47210 149065 4151384 6171602 1535885 664366 2429264 46686 149056 1340407 387921 5130900']
['N5S3KZ14ZXMB', '456dc9802f40f', 1541709810, 12, '67574 61340 71368 64197 76413 74466 75584 1082504 75819 71601 1105080 59668 61338 68770 2618132 377196 3144924 1189110 73199 8768822 4440410 75600 3137155 9842646 9634388']
['E8XM7XN0I9ZP', 'bd7024023b50c', 1541681730, 1, '2640816 1479953 1102870 1263953 1585673 1177492 4907426 2060466 1584259 2295118 2684315 1626019 4039268 3539198 2878378 1277265 2295636 2520084 3539160 4542787 4103980 3539196 8214704 4671846 5164320']
['7ONWT34CN25E', '3c079a841e3f4', 1541685604, 8, '1515971 1298859 1261215 1781079 1306142 1202646 1826865 2600968 2601853 1419500 1836725 2715934 2716218 2760682 2195990 2599402 4197574']
['1M11LKFLMSI2', 'a6e297c522591', 1541682487, 1, '40660 7500 7499 1322948 94038 115576 7490 971919 100353 51

['Y20ZV3ERA27L', '70198cf95b6b5', 1541623569, 10, '12606 46509 149298 46281 915127 3236596 12590 1362780 931109 47274 6733562 8524958 1312302 1151597 47357 1840113']
['O580387H291U', '35617c194b9d5', 1541622753, 1, '1321722 12278 119571 11903 18208 48816 107091 126844 49077 82475 99848 100020 4637846 99846 11946 99859 3825978 50187 12171 11934 11926 6626590 48809 50449 3823284']
['FH03J8IJQYST', 'ac069f31db6ee', 1541685505, 12, '9523164 8979342 6449334 7702928 1550985 1014297 4953486 1014231 1014225 3888054 4820218 1014275 1014259 106485 2314914 1014237 2807586 3436900']
['8288E3E322RU', 'c9b85a763b304', 1541603884, 5, '3583444 886065 3812468 5118742 3253104 100632 674856 84273 7988478 860391 1249379 105989 1032590 107291 100636 1073850 996545 5723312 105861 9607768 2239238 9773310 84443 42311 83049']
['2X854AV4PMX6', '4a8175f9fc982', 1541652470, 3, '73207 78658 72304 75050 4794668 3224686 62729 5703874 392796 60595 5100594 1198038 71051']
['WGEX9D32P638', 'd2aa466d2cf0d', 1541629343, 

In [252]:
my_sub = pd.DataFrame(submission_list[1:], columns=['user_id', 'session_id', 'timestamp', 'step', 'item_recommendations'])

In [262]:
my_sub.to_csv("./result.csv", index=False)